In [ ]:
!pip install dalex

In [ ]:
import pandas as pd
import pickle

# Metoda Break Down

![](figures/1.png)

*Prawdopodobnie najczęściej zadawanym pytaniem podczas próby zrozumienia przewidywań modelu dla pojedynczej obserwacji jest: które zmienne najbardziej przyczyniają się do tego wyniku?*

## Idea

![](figures/2.png)
##### http://ema.drwhy.ai/breakDown.html

#### Wyjaśnienie 1

Rozważamy zmienne objaśniające w kolejności płeć, klasę i wiek.
Zauważamy ujemny wkład dla dwóch pierwszych zmiennych i dodatni dla trzeciej. Fakt, że pasażer by chłopcem zmniejsza szanse przeżycia w porównaniu ze średnią przewidywaną przez model. Podróżował on w drugiej klasie, co jeszcze bardziej obniża prawdopodobieństwo przeżycia. Jednak to, że jest młody znacznie zwiększa jego prawdopodobieństwo przeżycia - ten wniosek wynika z faktu, że większość pasażerów drugiej klasy stanowili dorośli.



#### Wyjaśnienie 2

Rozważamy teraz zmienne w kolejności płeć, wiek i klasa. Rysunek wskazuje na pozytywny wkład klasy, inaczej niż w pierwszym wyjaśnieniu. Pasażer był chłopcem, ponownie widzimy, że zmiejsza to szanse. Jednak był bardzo młody, a to zwiększa szanse przeżycia w porównaniu z dorosłymi mężczyznami. Fakt, że podróżował w drugiej klasie również zwiększa szansę na przeżycie. Wynika to z faktu, że więkość dzieci podróżowała trzecią klasę.

![](figures/3.png)

### Plusy i minusy

- Metoda Break Down działa niezależnie od modelu, zatem możemy stosować do każdego modelu predykcyjnego, który zwraca predykcję dla pojedynczej obserwacji (instancji).

- Wykresy są łatwe w zrozumieniu, podejście sprowadza się do intuicyjnej interpretacji dla modeli liniowych.

- Złożoność algorytmu jest liniowa w stosunku do liczby zmiennych objaśniających.

- Wykresy mogą być mylące dla modeli zawierających interakacje. 

## Break Down na przykładzie wyceny nieruchomości

### Dane i modele 

#### Dane

In [ ]:
train = pd.read_csv("train.csv")

In [ ]:
train.head()

In [ ]:
variables = ["bedrooms", "bathrooms", "floors", "waterfront", "view", "condition", "grade", "lat", "long", "age", "m2_living", "m2_lot", "m2_basement"]

In [ ]:
X_train = train[variables]
y_train = train.price_log

In [ ]:
X_train.head()

#### Modele

In [ ]:
reg_lin = pickle.load(open("reg_lin", 'rb')) #regresja liniowa
reg_dd = pickle.load(open("reg_dd", 'rb')) #drzewo decyzyjne
reg_rr = pickle.load(open("reg_rr", 'rb')) #las losowy
reg_gb = pickle.load(open("reg_gb", 'rb')) #gradient boosting

### Wyjaśnienia

In [ ]:
import dalex as dx
import numpy as np

In [ ]:
def predict_function(model, data):
    return np.exp(model.predict(data))

In [ ]:
explainer = dx.Explainer(reg_rr, X_train, y_train, predict_function=predict_function)

*Explainer* jest obiektem, który opakowuje model i tworzy jednolitą strukturę.

#### predict_parts()

In [ ]:
X_train.iloc[0,:]

In [ ]:
explainer.predict(X_train)[0]

In [ ]:
pp = explainer.predict_parts(X_train.iloc[0,:])

In [ ]:
pp

In [ ]:
pp.plot()

# Metoda Shapley values

![](figures/4.png)

Metoda Shapley values jest uśrednieniem wszystkich ułożeń zmiennych.

![](figures/5.png)

Pozytywne i negatywne kontrybucje, boxploty informują o wartościach kontrybucji poszczególych ułożeń zmiennych.

### Plusy i minusy

- Wartości Shapleya zapewniają jednolite podejście do dekomponowania przewidywań modelu na wkłady, które mogą być przypisane addytywnie do różnych zmiennych objaśniających. 

- Ważnym praktycznym ograniczeniem ogólnej metody niezależnej od modelu jest to, że w przypadku dużych modeli obliczanie wartości Shapleya jest czasochłonne.

In [ ]:
pp_shap = explainer.predict_parts(X_train.iloc[0,:], type='shap')

In [ ]:
pp_shap

In [ ]:
pp_shap.plot()